In [1]:
import os
from dotenv import load_dotenv
import pandas as pd
import requests
import numpy as np
# Carregando as variáveis de ambiente do arquivo .env
load_dotenv()

True

In [2]:
ano = "2022"

In [3]:
from data_master_eng_ml.utils.twitch_api import build_query,fetch_data_with_pagination

2024-09-07 13:24:32.240 | INFO     | data_master_eng_ml.config:<module>:11 - PROJ_ROOT path is: /Users/josepaulo/data_master_eng_ml


In [4]:
URL_TWITCH_BASE = "https://api.igdb.com/v4"

In [5]:
# Fighting - 4
# Racing - 10
# 6	Full Release

In [36]:
from data_master_eng_ml.utils.mappings import (
    region_mapping_inverted,
    plataform_mapping,
    player_perspectives_mapping,
    genres_mapping,
    game_modes_mapping,
    age_rating_mapping,
    age_order
)

ImportError: cannot import name 'age_order' from 'data_master_eng_ml.utils.mappings' (/Users/josepaulo/data_master_eng_ml/data_master_eng_ml/utils/mappings.py)

In [9]:
# Exemplo de uso
url = f"{URL_TWITCH_BASE}/release_dates"

# Define os campos a serem selecionados
fields = [
    "category",
    "created_at",
    "date",
    "game",
    "human",
    "platform",
    "region",
    "status",
    "updated_at",
    "y",
]

# Define os filtros a serem aplicados
filters = {"status": "= 6", "y": f"= {ano}", "region": f"= 8"}

# Chamada da função com paginação
data_frame_games_find = fetch_data_with_pagination(url, build_query, fields, filters)
data_frame_games_find["region_name"] = data_frame_games_find["region"].map(
    region_mapping_inverted
)


In [11]:
games_id = list(set(data_frame_games_find["game"].to_list()))

In [12]:
games_id_list = ",".join(map(str, games_id))

In [13]:
len(games_id_list)

12720

In [14]:
# Exemplo de uso
url = f"{URL_TWITCH_BASE}/involved_companies"

# Define os campos a serem selecionados
fields = ["*"]
# Define os filtros a serem aplicados
filters = {
    "game": f"= ({games_id_list})",
}

# Chamada da função com paginação
data_frame_companies_find = fetch_data_with_pagination(
    url, build_query, fields, filters
)
data_frame_companies_find
company_id = list(set(data_frame_companies_find["company"].to_list()))
company_id_list = ",".join(map(str, company_id))

In [15]:
data_frame_companies_find

,id,company,created_at,developer,game,porting,publisher,supporting,updated_at,checksum
0,42850,3951,1481500800,True,14016,False,False,False,1481587200,4019ec1f-ed4f-42e2-e693-f40326782648
1,257101,22440,1709880472,True,227913,False,True,False,1710125858,c611d8b7-c58c-114f-d621-30f32ff9ae85
2,185685,12177,1662098659,False,27366,False,True,False,1662101178,c558368e-2c76-3a19-340d-6fc44364338d
3,258679,53624,1710720793,True,219363,False,True,False,1710722577,aec26cb7-5a2c-a090-d8c9-f4e25f3714ae
4,71561,16916,1542153600,True,105594,False,True,False,1542153600,54ce12d4-13cd-f897-3327-659d3193c279
...,...,...,...,...,...,...,...,...,...,...
1700,285699,33166,1725569281,False,173902,False,True,False,1725570717,b6f132b1-c755-d02c-1543-42736e03884c
1701,285697,41282,1725569280,True,173902,False,False,False,1725570717,552b192b-b770-01d7-70aa-204dae6150a9
1702,285698,8891,1725569281,False,173902,False,True,False,1725570717,e27027f4-c7e7-6338-fdf4-4f1d35c8dc24
1703,285753,12186,1725585838,False,204360,False,True,False,1725585846,e6799aaa-60f7-7810-3f74-f7271cc0e37b


In [16]:
import pycountry_convert as pc
import pycountry


def country_to_continent(country_code):
    try:
        # Converte o código do país para o nome do continente
        country = pycountry.countries.get(numeric=str(country_code))
        continent_code = pc.country_alpha2_to_continent_code(country.alpha_2)
        continent_name = pc.convert_continent_code_to_continent_name(continent_code)
        return continent_name
    except:
        return "Unknown"  # Retorna 'Unknown' se o código do país não for reconhecido

In [17]:
def array_count(value):
    if value == "Unknown":
        return -1
    else:
        return len(value)

In [18]:
# Exemplo de uso
url = f"{URL_TWITCH_BASE}/companies"

# Define os campos a serem selecionados
fields = [
    "developed",
    "slug",
    "published",
    "country",
    "start_date",
    "start_date_category",
    "parent",
]
# Define os filtros a serem aplicados
filters = {
    "id": f"= ({company_id_list})",
}
# Chamada da função com paginação
data_frame_companies = fetch_data_with_pagination(url, build_query, fields)
data_frame_companies["developed"] = data_frame_companies["developed"].fillna("Unknown")
data_frame_companies["published"] = data_frame_companies["published"].fillna("Unknown")
data_frame_companies["country"] = data_frame_companies["country"].fillna(1)
data_frame_companies["country"] = data_frame_companies["country"].astype(int)
data_frame_companies["games_developed"] = data_frame_companies["developed"].apply(
    array_count
)
data_frame_companies["has_parents"] = data_frame_companies["parent"].notna().astype(int)
data_frame_companies["games_published"] = data_frame_companies["published"].apply(
    array_count
)
data_frame_companies["continent_name"] = data_frame_companies["country"].apply(
    country_to_continent
)

data_frame_companies = data_frame_companies.drop(
    columns=["developed", "published", "country", "parent", "start_date_category"]
)

In [19]:
# Inner Join (default)
companies_join = pd.merge(
    data_frame_companies_find,
    data_frame_companies,
    left_on="company",
    right_on="id",
    how="inner",
)

In [20]:
df_companies_final = companies_join[
    ["game", "games_developed", "has_parents", "games_published", "continent_name"]
]

In [21]:
# Exemplo de uso
url = f"{URL_TWITCH_BASE}/multiplayer_modes"

# Define os campos a serem selecionados
fields = [
    "campaigncoop",
    "game",
    "lancoop",
    "offlinecoop",
    "offlinecoopmax",
    "offlinemax",
    "onlinecoop",
    "onlinecoopmax",
    "onlinemax",
    "splitscreen",
]
# Define os filtros a serem aplicados
filters = {
    "game": f"= ({games_id_list})",
}

# Chamada da função com paginação
data_frame_multiplayer_modes = fetch_data_with_pagination(
    url, build_query, fields, filters
)
# Converte todas as colunas booleanas em 0/1
boolean_columns = data_frame_multiplayer_modes.select_dtypes(include='bool').columns
data_frame_multiplayer_modes[boolean_columns] = data_frame_multiplayer_modes[boolean_columns].astype(int)

data_frame_multiplayer_modes

,id,campaigncoop,game,lancoop,offlinecoop,offlinecoopmax,offlinemax,onlinecoop,onlinecoopmax,onlinemax,splitscreen
0,5456,0,13205,0,0,0.0,0.0,1,4.0,0.0,0
1,8032,0,34894,0,0,0.0,0.0,1,0.0,16.0,0
2,10346,0,131957,0,0,NaN,NaN,0,NaN,NaN,0
3,10862,0,105594,0,0,NaN,NaN,1,NaN,NaN,0
4,11594,0,72,1,1,2.0,0.0,1,2.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...
143,28661,0,231049,0,1,4.0,4.0,0,NaN,NaN,1
144,28772,0,40971,0,0,NaN,2.0,0,NaN,NaN,0
145,28771,0,40971,0,0,NaN,2.0,0,NaN,NaN,0
146,28847,0,307264,0,0,NaN,2.0,0,NaN,2.0,0


In [23]:
# Exemplo de uso
url = f"{URL_TWITCH_BASE}/games"

# Define os campos a serem selecionados
fields = [
    "name",
    "game_modes",
    "genres",
    "age_ratings",
    "involved_companies",
    "player_perspectives",
    "platforms",
    "rating",
    "remasters",
]

# Define os filtros a serem aplicados
filters = {
    "id": f"= ({games_id_list})",
    "category": f"= (0)",
}

# Chamada da função com paginação
data_frame_games = fetch_data_with_pagination(url, build_query, fields, filters)

# Trata as colunas
data_frame_games["player_perspectives"] = data_frame_games[
    "player_perspectives"
].fillna("Unknown")
data_frame_games["game_modes"] = data_frame_games["game_modes"].fillna("unknown_game_mode")
data_frame_games["genres"] = data_frame_games["genres"].fillna("unknown_genres_name")
data_frame_games["player_perspective_name"] = (
    data_frame_games["player_perspectives"]
    .map(lambda x: [player_perspectives_mapping.get(i, "unknown_player_perspectives") for i in x])
    .apply(lambda x: list(set(x)))
)
data_frame_games["platforms_name"] = (
    data_frame_games["platforms"]
    .map(lambda x: [plataform_mapping.get(i, "unknown_platforms_name") for i in x])
    .apply(lambda x: list(set(x)))
)
data_frame_games["genres_first"] = (
    data_frame_games["genres"]
    .map(lambda x: [genres_mapping.get(i, "unknown_genres_name") for i in x])
    .apply(lambda x: list(set(x))).apply(lambda x: x[0])
)
data_frame_games["game_modes_name"] = (
    data_frame_games["game_modes"]
    .map(lambda x: [game_modes_mapping.get(i, "unknown_game_mode") for i in x])
    .apply(lambda x: list(set(x)))
)
data_frame_games["has_remaster"] = data_frame_games["remasters"].notna()
data_frame_games["target"] = np.where(data_frame_games["rating"].isna(),0,1)
data_frame_games = data_frame_games.drop(
    columns=[
        "game_modes",
        "player_perspectives",
        "remasters",
        "genres",
        "platforms",
        "involved_companies",
        "rating",
        
    ]
)


In [24]:
data_frame_games

,id,age_ratings,name,player_perspective_name,platforms_name,genres_first,game_modes_name,has_remaster,target
0,71,"[984, 4395, 47678, 47679, 47680, 47681, 47682]",Portal,[first-person],"[pc, modern_console, mobile]",shooter,[single-player],False,1
1,72,"[11721, 32022, 47684, 47685, 47687, 125644, 14...",Portal 2,[first-person],"[pc, modern_console]",shooter,"[split-screen, single-player, multiplayer, co-...",False,1
2,4250,"[65229, 142924, 179089]",Kingdom: The Far Reaches,[third-person],"[pc, less_common_portable_console, others, unk...",adventure,[single-player],False,0
3,11145,"[92695, 137576]",Battle Kid: Fortress of Peril,[side-view],"[modern_console, classic_console]",shooter,[single-player],False,0
4,13205,"[44832, 68971, 86979, 91946, 110632]",Astroneer,"[third-person, first-person]","[pc, modern_console]",simulator,"[single-player, multiplayer, co-operative]",False,1
...,...,...,...,...,...,...,...,...,...
1506,61080,"[45079, 59140, 97948, 119399, 171971, 171972, ...",Ninja Village,[bird-view-slash-isometric],"[pc, modern_console, mobile]",strategy,[single-player],False,0
1507,132058,"[82252, 102582, 111285, 114570]",Rogue Legacy 2,[side-view],"[pc, modern_console]",adventure,[single-player],False,1
1508,143612,"[56633, 90852, 118403, 180772, 180773, 180774,...",Neon White,"[text, first-person]","[pc, modern_console]",shooter,[single-player],False,1
1509,135929,"[78029, 78068, 130323, 144094, 144095, 144096]",Apico,[bird-view-slash-isometric],"[pc, modern_console]",simulator,"[single-player, multiplayer, co-operative]",False,0


In [26]:
# Definir uma ordem para os grupos etários
age_order = pd.CategoricalDtype(
    categories=[
        "Rating Pending",
        "All Ages",
        "3+",
        "6+",
        "10+",
        "12+",
        "14+",
        "15+",
        "16+",
        "18+",
    ],
    ordered=True,
)

In [27]:
def age_classif(list):
    url = f"{URL_TWITCH_BASE}/age_ratings"

    try:
        lista_query = ",".join(map(str, list))
        # Define os campos a serem selecionados
        fields = ["rating"]

        filters = {
            "id": f"= ({lista_query})",
        }
        # Chamada da função com paginação
        data_frame = fetch_data_with_pagination(url, build_query, fields, filters)
        # # Exemplo de uso no DataFrame
        data_frame["age_rating_group"] = data_frame["rating"].map(age_rating_mapping)
        # Convertendo a coluna 'age_rating_group' para essa ordem
        data_frame["age_rating_group"] = data_frame["age_rating_group"].astype(
            age_order
        )
        # Pegar o maior valor de 'age_rating_group'
        max_age_rating_group = data_frame["age_rating_group"].max()
        return max_age_rating_group
    except:
        return "No Rating"

In [28]:
data_frame_games["age_classif"] = data_frame_games["age_ratings"].apply(age_classif).drop(columns=["age_ratings",])

In [29]:
data_frame_games =  data_frame_games.drop(columns=["age_ratings"])


In [30]:
data_frame_games

,id,name,player_perspective_name,platforms_name,genres_first,game_modes_name,has_remaster,target,age_classif
0,71,Portal,[first-person],"[pc, modern_console, mobile]",shooter,[single-player],False,1,15+
1,72,Portal 2,[first-person],"[pc, modern_console]",shooter,"[split-screen, single-player, multiplayer, co-...",False,1,12+
2,4250,Kingdom: The Far Reaches,[third-person],"[pc, less_common_portable_console, others, unk...",adventure,[single-player],False,0,12+
3,11145,Battle Kid: Fortress of Peril,[side-view],"[modern_console, classic_console]",shooter,[single-player],False,0,3+
4,13205,Astroneer,"[third-person, first-person]","[pc, modern_console]",simulator,"[single-player, multiplayer, co-operative]",False,1,6+
...,...,...,...,...,...,...,...,...,...
1506,61080,Ninja Village,[bird-view-slash-isometric],"[pc, modern_console, mobile]",strategy,[single-player],False,0,12+
1507,132058,Rogue Legacy 2,[side-view],"[pc, modern_console]",adventure,[single-player],False,1,12+
1508,143612,Neon White,"[text, first-person]","[pc, modern_console]",shooter,[single-player],False,1,15+
1509,135929,Apico,[bird-view-slash-isometric],"[pc, modern_console]",simulator,"[single-player, multiplayer, co-operative]",False,0,3+


In [31]:
data_frame_games_find[["game", "region_name",]]

df_companies_final
data_frame_multiplayer_modes

,id,campaigncoop,game,lancoop,offlinecoop,offlinecoopmax,offlinemax,onlinecoop,onlinecoopmax,onlinemax,splitscreen
0,5456,0,13205,0,0,0.0,0.0,1,4.0,0.0,0
1,8032,0,34894,0,0,0.0,0.0,1,0.0,16.0,0
2,10346,0,131957,0,0,NaN,NaN,0,NaN,NaN,0
3,10862,0,105594,0,0,NaN,NaN,1,NaN,NaN,0
4,11594,0,72,1,1,2.0,0.0,1,2.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...
143,28661,0,231049,0,1,4.0,4.0,0,NaN,NaN,1
144,28772,0,40971,0,0,NaN,2.0,0,NaN,NaN,0
145,28771,0,40971,0,0,NaN,2.0,0,NaN,NaN,0
146,28847,0,307264,0,0,NaN,2.0,0,NaN,2.0,0


In [32]:
data_frame_games_find.drop_duplicates(subset="game")

,id,category,created_at,date,game,human,platform,region,updated_at,y,status,region_name
0,594406,0,1715767847,1645574400,139339,"Feb 23, 2022",34,8,1715907478,2022,6,worldwide
3,594525,0,1715836304,1652832000,200839,"May 18, 2022",48,8,1715916873,2022,6,worldwide
4,594526,0,1715836308,1663804800,219337,"Sep 22, 2022",6,8,1715916888,2022,6,worldwide
9,594868,0,1715924700,1649289600,301306,"Apr 07, 2022",6,8,1715931145,2022,6,worldwide
10,594710,0,1715900186,1647388800,301158,"Mar 16, 2022",24,8,1715973541,2022,6,worldwide
...,...,...,...,...,...,...,...,...,...,...,...,...
3786,593435,0,1715385461,1645142400,191747,"Feb 18, 2022",14,8,1715631637,2022,6,worldwide
3787,593729,0,1715482056,1665619200,221696,"Oct 13, 2022",130,8,1715650510,2022,6,worldwide
3790,594124,0,1715728597,1651104000,197890,"Apr 28, 2022",6,8,1715728620,2022,6,worldwide
3791,594570,0,1715850330,1652832000,200565,"May 18, 2022",6,8,1715850341,2022,6,worldwide


In [33]:
df_companies_games = pd.merge(
    df_companies_final,
    data_frame_games_find[["game", "region_name",]].drop_duplicates(subset="game"),
    on="game",
    how="inner",
)
df_companies_games

,game,games_developed,has_parents,games_published,continent_name,region_name
0,14016,22,0,-1,Unknown,worldwide
1,227913,11,0,3,Unknown,worldwide
2,27366,1,0,1,North America,worldwide
3,219363,1,0,1,Unknown,worldwide
4,105594,1,0,1,Unknown,worldwide
...,...,...,...,...,...,...
1700,173902,-1,0,28,Europe,worldwide
1701,173902,1,0,-1,Unknown,worldwide
1702,173902,45,0,49,Unknown,worldwide
1703,204360,-1,1,25,Europe,worldwide


In [34]:
df_companies_games_multiplayer = pd.merge(
    df_companies_games.drop_duplicates(subset="game"),
    data_frame_multiplayer_modes[
        ["game", "onlinecoop", "onlinecoopmax", "onlinemax", "splitscreen"]
    ],
    on="game",
    how="left",
)
df_companies_games_multiplayer

,game,games_developed,has_parents,games_published,continent_name,region_name,onlinecoop,onlinecoopmax,onlinemax,splitscreen
0,14016,22,0,-1,Unknown,worldwide,NaN,NaN,NaN,NaN
1,227913,11,0,3,Unknown,worldwide,NaN,NaN,NaN,NaN
2,27366,1,0,1,North America,worldwide,NaN,NaN,NaN,NaN
3,219363,1,0,1,Unknown,worldwide,NaN,NaN,NaN,NaN
4,105594,1,0,1,Unknown,worldwide,1.0,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...
1226,314578,1,0,1,Unknown,worldwide,NaN,NaN,NaN,NaN
1227,314580,2,0,2,Unknown,worldwide,NaN,NaN,NaN,NaN
1228,207093,6,0,285,North America,worldwide,NaN,NaN,NaN,NaN
1229,173902,-1,0,28,Europe,worldwide,NaN,NaN,NaN,NaN


In [35]:
data_frame_multiplayer_modes[
    ["game", "onlinecoop", "onlinecoopmax", "onlinemax", "splitscreen"]
]

,game,onlinecoop,onlinecoopmax,onlinemax,splitscreen
0,13205,1,4.0,0.0,0
1,34894,1,0.0,16.0,0
2,131957,0,NaN,NaN,0
3,105594,1,NaN,NaN,0
4,72,1,2.0,0.0,1
...,...,...,...,...,...
143,231049,0,NaN,NaN,1
144,40971,0,NaN,NaN,0
145,40971,0,NaN,NaN,0
146,307264,0,NaN,2.0,0


In [ ]:
df_join = pd.merge(
    data_frame_games,
    df_companies_games_multiplayer,
    left_on="id",
    right_on="game",
    how="left",
).drop(columns=['game']).drop_duplicates(subset="id")

In [ ]:
# Explode as listas de 'attributes' para transformar cada elemento em uma linha separada
df_exploded = df_join[['id',"platforms_name"]].explode('platforms_name')

# Converte a coluna 'attributes' em colunas de dummies, com prefixo para evitar conflitos
dummies = pd.get_dummies(df_exploded['platforms_name'].str.replace('-', '_'))

# Agrupa por 'id' e faz a soma para agrupar as flags 0/1
dummies = dummies.groupby(df_exploded['id']).sum().reset_index()

# Junte os dummies ao DataFrame original
df_join = df_join.drop(columns=['platforms_name']).merge(dummies, on='id')


In [ ]:
# Explode as listas de 'attributes' para transformar cada elemento em uma linha separada
df_exploded = df_join[['id',"game_modes_name"]].explode('game_modes_name')

# Converte a coluna 'attributes' em colunas de dummies, com prefixo para evitar conflitos
dummies = pd.get_dummies(df_exploded['game_modes_name'].str.replace('-', '_'))

# Agrupa por 'id' e faz a soma para agrupar as flags 0/1
dummies = dummies.groupby(df_exploded['id']).sum().reset_index()

# Junte os dummies ao DataFrame original
df_join = df_join.drop(columns=['game_modes_name']).merge(dummies, on='id')


In [ ]:
# Explode as listas de 'attributes' para transformar cada elemento em uma linha separada
df_exploded = df_join[['id',"player_perspective_name"]].explode('player_perspective_name')

# Converte a coluna 'attributes' em colunas de dummies, com prefixo para evitar conflitos
dummies = pd.get_dummies(df_exploded['player_perspective_name'].str.replace('-', '_'))

# Agrupa por 'id' e faz a soma para agrupar as flags 0/1
dummies = dummies.groupby(df_exploded['id']).sum().reset_index()

# Junte os dummies ao DataFrame original
df_join = df_join.drop(columns=['player_perspective_name']).merge(dummies, on='id')


In [ ]:
# Converte todas as colunas booleanas em 0/1
boolean_columns = df_join.select_dtypes(include='bool').columns
df_join[boolean_columns] = df_join[boolean_columns].astype(int)


In [ ]:
import numpy as np

In [ ]:
df_join['has_global_launch'] = np.where(df_join['region_name'] == 'worldwide',1,0)

In [ ]:
df_join = df_join.drop(columns=['region_name'])

In [ ]:
df_join

,id,name,genres_first,has_remaster,target,age_classif,games_developed,has_parents,games_published,continent_name,...,unknown_game_mode,auditory,bird_view_slash_isometric,first_person,side_view,text,third_person,unknown_player_perspectives,virtual_reality,has_global_launch
0,71,Portal,shooter,0,1,15+,125.0,0.0,1346.0,North America,...,0,0,0,1,0,0,0,0,0,1
1,72,Portal 2,shooter,0,1,12+,86.0,0.0,94.0,North America,...,0,0,0,1,0,0,0,0,0,1
2,1344,Wild Guns,arcade,1,1,12+,12.0,0.0,36.0,Europe,...,0,0,0,0,0,0,1,0,0,0
3,4250,Kingdom: The Far Reaches,adventure,0,0,12+,4.0,0.0,39.0,Unknown,...,0,0,0,0,0,0,1,0,0,1
4,6614,Zero Escape: Virtue's Last Reward,visual-novel,0,1,18+,1.0,0.0,132.0,Europe,...,0,0,0,1,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1300,225446,Dropship Commander,arcade,0,0,All Ages,3.0,0.0,3.0,North America,...,0,0,0,0,1,0,0,0,1,1
1301,250264,Tokyo Pinball,arcade,0,0,No Rating,1.0,0.0,1.0,Unknown,...,0,0,1,0,0,0,0,0,0,1
1302,307160,Tanuki Tales,arcade,0,0,No Rating,1.0,0.0,-1.0,Unknown,...,0,0,0,0,0,0,1,0,0,1
1303,229164,Cubic Light,indie,0,0,3+,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
df_join.to_csv(f'twitch_api_data_{ano}.csv',header=True,index=False)